In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/deliveries_updated_mens_ipl_upto_2024.csv')
df.head()

,matchId,inning,over_ball,over,ball,batting_team,bowling_team,batsman,non_striker,bowler,batsman_runs,extras,isWide,isNoBall,Byes,LegByes,Penalty,dismissal_kind,player_dismissed,date
0,335982,1,0.1,0,1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,P Kumar,0,1,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2008-04-18
1,335982,1,0.2,0,2,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008-04-18
2,335982,1,0.3,0,3,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2008-04-18
3,335982,1,0.4,0,4,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008-04-18
4,335982,1,0.5,0,5,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008-04-18


In [ ]:
# Columns that should be integers
columns_to_fix = ['isNoBall', 'Byes', 'LegByes', 'Penalty', 'isWide']

# Replace NaNs with 0 and convert to int
for col in columns_to_fix:
    df[col] = df[col].fillna(0).astype(int)

# Check if data types have changed
print(df[columns_to_fix].dtypes)


isNoBall    int64
Byes        int64
LegByes     int64
Penalty     int64
isWide      int64
dtype: object


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260920 entries, 0 to 260919
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   matchId           260920 non-null  int64  
 1   inning            260920 non-null  int64  
 2   over_ball         260920 non-null  float64
 3   over              260920 non-null  int64  
 4   ball              260920 non-null  int64  
 5   batting_team      260920 non-null  object 
 6   bowling_team      260920 non-null  object 
 7   batsman           260920 non-null  object 
 8   non_striker       260920 non-null  object 
 9   bowler            260920 non-null  object 
 10  batsman_runs      260920 non-null  int64  
 11  extras            260920 non-null  int64  
 12  isWide            260920 non-null  int64  
 13  isNoBall          260920 non-null  int64  
 14  Byes              260920 non-null  int64  
 15  LegByes           260920 non-null  int64  
 16  Penalty           26

In [ ]:

# Filter only middle overs (1 to 6)
powerplay_batting = df[(df['over'] >= 1) & (df['over'] <= 6)]

# Group by batsman and compute stats
batter_stats = powerplay_batting.groupby('batsman').agg(
    Runs=('batsman_runs', 'sum'),
    Balls_Faced=('batsman_runs', 'count'),
    Fours=('batsman_runs', lambda x: (x == 4).sum()),
    Sixes=('batsman_runs', lambda x: (x == 6).sum()),
    Dismissals=('player_dismissed', lambda x: x.notna().sum())
).reset_index()

# Convert Fours and Sixes to integers
batter_stats['Fours'] = batter_stats['Fours'].astype(int)
batter_stats['Sixes'] = batter_stats['Sixes'].astype(int)

# Add computed metrics
batter_stats['Strike Rate'] = (batter_stats['Runs'] / batter_stats['Balls_Faced']) * 100
batter_stats['Boundary Rate'] = (batter_stats['Balls_Faced']/(batter_stats['Fours'] + batter_stats['Sixes']))
batter_stats['Average'] = batter_stats.apply(
    lambda row: row['Runs'] / row['Dismissals'] if row['Dismissals'] > 0 else row['Runs'], axis=1
)

# Round values
batter_stats = batter_stats.round(2)

# Sort by Runs and select top 15
top_15 = batter_stats.sort_values(by='Runs', ascending=False).head(15)

top_15


,batsman,Runs,Balls_Faced,Fours,Sixes,Dismissals,Strike Rate,Boundary Rate,Average
277,S Dhawan,3320,2726,416,78,100,121.79,5.52,33.20
77,DA Warner,3171,2233,395,109,63,142.01,4.43,50.33
343,V Kohli,2699,2344,303,64,70,115.15,6.39,38.56
68,CH Gayle,2314,1667,239,153,68,138.81,4.25,34.03
105,G Gambhir,2244,1834,299,35,70,122.36,5.49,32.06
103,F du Plessis,2215,1563,261,82,50,141.71,4.56,44.30
30,AM Rahane,2212,1806,266,48,69,122.48,5.75,32.06
270,RV Uthappa,2082,1671,257,57,66,124.60,5.32,31.55
258,RG Sharma,2082,1733,217,91,76,120.14,5.63,27.39
153,KL Rahul,1979,1552,201,75,33,127.51,5.62,59.97


In [ ]:
# Filter only powerplay overs (1 to 6)
Middleovers_batting = df[(df['over'] >= 7) & (df['over'] <= 15)]

# Group by batsman and compute stats
batter_stats = Middleovers_batting.groupby('batsman').agg(
    Runs=('batsman_runs', 'sum'),
    Balls_Faced=('batsman_runs', 'count'),
    Fours=('batsman_runs', lambda x: (x == 4).sum()),
    Sixes=('batsman_runs', lambda x: (x == 6).sum()),
    Dismissals=('player_dismissed', lambda x: x.notna().sum())
).reset_index()

# Convert Fours and Sixes to integers
batter_stats['Fours'] = batter_stats['Fours'].astype(int)
batter_stats['Sixes'] = batter_stats['Sixes'].astype(int)

# Add computed metrics
batter_stats['Strike Rate'] = (batter_stats['Runs'] / batter_stats['Balls_Faced']) * 100
batter_stats['Boundary Rate'] = (batter_stats['Balls_Faced']/(batter_stats['Fours'] + batter_stats['Sixes']))
batter_stats['Average'] = batter_stats.apply(
    lambda row: row['Runs'] / row['Dismissals'] if row['Dismissals'] > 0 else row['Runs'], axis=1
)

# Round values
batter_stats = batter_stats.round(2)

# Sort by Runs and select top 15
top_15 = batter_stats.sort_values(by='Runs', ascending=False).head(15)

top_15

,batsman,Runs,Balls_Faced,Fours,Sixes,Dismissals,Strike Rate,Boundary Rate,Average
516,V Kohli,3737,2930,260,127,96,127.54,7.57,38.93
391,RG Sharma,3017,2466,246,99,96,122.34,7.15,31.43
444,SK Raina,2999,2260,227,114,97,132.70,6.63,30.92
25,AB de Villiers,2808,2032,191,120,62,138.19,6.53,45.29
229,KD Karthik,2696,2211,248,62,103,121.94,7.13,26.17
54,AT Rayudu,2510,2012,181,101,82,124.75,7.13,30.61
122,DA Warner,2504,1816,172,101,66,137.89,6.65,37.94
419,S Dhawan,2494,1968,234,54,71,126.73,6.83,35.13
305,MS Dhoni,2349,2241,140,75,54,104.82,10.42,43.50
469,SV Samson,2335,1703,142,115,73,137.11,6.63,31.99


In [ ]:
# Filter only Death overs (16 to 20)
Deathovers_df = df[(df['over'] >= 16) & (df['over'] <= 20)]

# Group by batsman and compute stats
death_batting = Deathovers_df.groupby('batsman').agg(
    Runs=('batsman_runs', 'sum'),
    Balls_Faced=('batsman_runs', 'count'),
    Fours=('batsman_runs', lambda x: (x == 4).sum()),
    Sixes=('batsman_runs', lambda x: (x == 6).sum()),
    Dismissals=('player_dismissed', lambda x: x.notna().sum())
).reset_index()

# Convert Fours and Sixes to integers
batter_stats['Fours'] = batter_stats['Fours'].astype(int)
batter_stats['Sixes'] = batter_stats['Sixes'].astype(int)

# Add computed metrics
batter_stats['Strike Rate'] = (batter_stats['Runs'] / batter_stats['Balls_Faced']) * 100
batter_stats['Boundary Rate'] = (batter_stats['Balls_Faced']/(batter_stats['Fours'] + batter_stats['Sixes']))
batter_stats['Average'] = batter_stats.apply(
    lambda row: row['Runs'] / row['Dismissals'] if row['Dismissals'] > 0 else row['Runs'], axis=1
)

# Round values
batter_stats = batter_stats.round(2)

# Sort by Runs and select top 15
top_15 = batter_stats.sort_values(by='Runs', ascending=False).head(15)

top_15


,batsman,Runs,Balls_Faced,Fours,Sixes,Dismissals,Strike Rate,Boundary Rate,Average
516,V Kohli,3737,2930,260,127,96,127.54,7.57,38.93
391,RG Sharma,3017,2466,246,99,96,122.34,7.15,31.43
444,SK Raina,2999,2260,227,114,97,132.70,6.63,30.92
25,AB de Villiers,2808,2032,191,120,62,138.19,6.53,45.29
229,KD Karthik,2696,2211,248,62,103,121.94,7.13,26.17
54,AT Rayudu,2510,2012,181,101,82,124.75,7.13,30.61
122,DA Warner,2504,1816,172,101,66,137.89,6.65,37.94
419,S Dhawan,2494,1968,234,54,71,126.73,6.83,35.13
305,MS Dhoni,2349,2241,140,75,54,104.82,10.42,43.50
469,SV Samson,2335,1703,142,115,73,137.11,6.63,31.99


In [ ]:
# Convert 'over' to numeric if it's not already
df['over'] = pd.to_numeric(df['over'], errors='coerce')

# Step 1: Filter Powerplay overs (1 to 6)
powerplay_bowling = df[(df['over'] >= 1) & (df['over'] <= 6)]

# Step 2: Group by bowler and calculate basic stats
grouped = powerplay_bowling.groupby('bowler').agg(
    balls_bowled=('bowler', 'size'),
    runs_given=('batsman_runs', 'sum'),
    extras=('extras', 'sum'),
    wickets_taken=('player_dismissed', lambda x: x.notnull().sum()),
    fours_given=('batsman_runs', lambda x: x[x == 4].count()),
    sixes_given=('batsman_runs', lambda x: x[x == 6].count()),
    dot_balls=('batsman_runs', lambda x: x[x == 0].count())
)

# Step 3: Derived metrics
grouped['total_runs_with_extras'] = grouped['runs_given'] + grouped['extras']

# Overs Bowled (like 4.3)
grouped['overs_bowled'] = grouped['balls_bowled'] // 6 + (grouped['balls_bowled'] % 6) / 10

# Economy rate
grouped['economy'] = (grouped['total_runs_with_extras'] / grouped['balls_bowled']) * 6

# Dot ball percentage
grouped['dot_ball_percentage'] = (grouped['dot_balls'] / grouped['balls_bowled']) * 100

# Bowling average
grouped['bowling_average'] = grouped['total_runs_with_extras'] / grouped['wickets_taken'].replace(0, np.nan)

# Step 4: Round to 2 decimal places
grouped['economy'] = grouped['economy'].round(2)
grouped['dot_ball_percentage'] = grouped['dot_ball_percentage'].round(2)
grouped['bowling_average'] = grouped['bowling_average'].round(2)

# Step 5: Reorder and rename columns
final_stats = grouped[[
    'overs_bowled', 'balls_bowled', 'total_runs_with_extras', 'wickets_taken',
    'economy', 'fours_given', 'sixes_given', 'dot_ball_percentage', 'bowling_average'
]]

final_stats.columns = [
    'Overs', 'Balls', 'Runs', 'Wkts', 'Econ', '4s', '6s', 'Dot%', 'Avg'
]

# Reset index to make bowler a column
final_stats = final_stats.reset_index()

# Show all columns in one row for top bowlers
print(final_stats.sort_values(by='Wkts', ascending=False).head(10).to_string(index=False))

         bowler  Overs  Balls  Runs  Wkts  Econ  4s  6s  Dot%   Avg
       I Sharma  230.4   1384  1727    59  7.49 235  38 53.90 29.27
 Sandeep Sharma  204.1   1225  1514    53  7.42 189  41 48.49 28.57
       UT Yadav  218.4   1312  1784    51  8.16 229  50 49.16 34.98
       R Ashwin  229.0   1374  1517    48  6.62 115  53 44.18 31.60
        B Kumar  244.2   1466  1715    48  7.02 198  50 52.93 35.73
      DL Chahar  154.5    929  1282    47  8.28 154  42 44.35 27.28
         Z Khan  157.1    943  1095    45  6.97 141  21 50.80 24.33
    DS Kulkarni  163.5    983  1274    42  7.78 176  29 49.14 30.33
 Mohammed Shami  180.5   1085  1461    41  8.08 194  40 49.95 35.63
Harbhajan Singh  157.5    947  1065    41  6.75  99  35 47.62 25.98


In [ ]:
# Convert 'over' to numeric if it's not already
df['over'] = pd.to_numeric(df['over'], errors='coerce')

# Step 1: Filter Powerplay overs (1 to 6)
Middleover_bowling = df[(df['over'] >= 7) & (df['over'] <= 16)]

# Step 2: Group by bowler and calculate basic stats
grouped = Middleover_bowling.groupby('bowler').agg(
    balls_bowled=('bowler', 'size'),
    runs_given=('batsman_runs', 'sum'),
    extras=('extras', 'sum'),
    wickets_taken=('player_dismissed', lambda x: x.notnull().sum()),
    fours_given=('batsman_runs', lambda x: x[x == 4].count()),
    sixes_given=('batsman_runs', lambda x: x[x == 6].count()),
    dot_balls=('batsman_runs', lambda x: x[x == 0].count())
)

# Step 3: Derived metrics
grouped['total_runs_with_extras'] = grouped['runs_given'] + grouped['extras']

# Overs Bowled (like 4.3)
grouped['overs_bowled'] = grouped['balls_bowled'] // 6 + (grouped['balls_bowled'] % 6) / 10

# Economy rate
grouped['economy'] = (grouped['total_runs_with_extras'] / grouped['balls_bowled']) * 6

# Dot ball percentage
grouped['dot_ball_percentage'] = (grouped['dot_balls'] / grouped['balls_bowled']) * 100

# Bowling average
grouped['bowling_average'] = grouped['total_runs_with_extras'] / grouped['wickets_taken'].replace(0, np.nan)

# Step 4: Round to 2 decimal places
grouped['economy'] = grouped['economy'].round(2)
grouped['dot_ball_percentage'] = grouped['dot_ball_percentage'].round(2)
grouped['bowling_average'] = grouped['bowling_average'].round(2)

# Step 5: Reorder and rename columns
final_stats = grouped[[
    'overs_bowled', 'balls_bowled', 'total_runs_with_extras', 'wickets_taken',
    'economy', 'fours_given', 'sixes_given', 'dot_ball_percentage', 'bowling_average'
]]

final_stats.columns = [
    'Overs', 'Balls', 'Runs', 'Wkts', 'Econ', '4s', '6s', 'Dot%', 'Avg'
]

# Reset index to make bowler a column
final_stats = final_stats.reset_index()

# Show all columns in one row for top bowlers
print(final_stats.sort_values(by='Wkts', ascending=False).head(10).to_string(index=False))

         bowler  Overs  Balls  Runs  Wkts  Econ  4s  6s  Dot%   Avg
      YS Chahal  446.2   2678  3406   158  7.63 181 162 36.63 21.56
      PP Chawla  503.2   3020  4012   152  7.97 258 183 34.74 26.39
       A Mishra  465.5   2795  3378   139  7.25 178 149 36.78 24.30
      RA Jadeja  513.4   3082  3876   133  7.55 216 168 33.84 29.14
       R Ashwin  495.3   2973  3505   127  7.07 178 130 34.38 27.60
    Rashid Khan  372.3   2235  2499   117  6.71 159  89 38.97 21.36
      SP Narine  409.0   2454  2694   116  6.59 148  89 38.02 23.22
Harbhajan Singh  387.1   2323  2777   104  7.17 177  99 35.77 26.70
       AR Patel  368.1   2209  2662    90  7.23 145 108 33.73 29.58
       DJ Bravo  320.3   1923  2431    82  7.59 196  64 35.31 29.65


In [ ]:
# Convert 'over' to numeric if it's not already
df['over'] = pd.to_numeric(df['over'], errors='coerce')

# Step 1: Filter Powerplay overs (1 to 6)
death_bowling = df[(df['over'] >= 17) & (df['over'] <= 20)]

# Step 2: Group by bowler and calculate basic stats
grouped = death_bowling.groupby('bowler').agg(
    balls_bowled=('bowler', 'size'),
    runs_given=('batsman_runs', 'sum'),
    extras=('extras', 'sum'),
    wickets_taken=('player_dismissed', lambda x: x.notnull().sum()),
    fours_given=('batsman_runs', lambda x: x[x == 4].count()),
    sixes_given=('batsman_runs', lambda x: x[x == 6].count()),
    dot_balls=('batsman_runs', lambda x: x[x == 0].count())
)

# Step 3: Derived metrics
grouped['total_runs_with_extras'] = grouped['runs_given'] + grouped['extras']

# Overs Bowled (like 4.3)
grouped['overs_bowled'] = grouped['balls_bowled'] // 6 + (grouped['balls_bowled'] % 6) / 10

# Economy rate
grouped['economy'] = (grouped['total_runs_with_extras'] / grouped['balls_bowled']) * 6

# Dot ball percentage
grouped['dot_ball_percentage'] = (grouped['dot_balls'] / grouped['balls_bowled']) * 100

# Bowling average
grouped['bowling_average'] = grouped['total_runs_with_extras'] / grouped['wickets_taken'].replace(0, np.nan)

# Step 4: Round to 2 decimal places
grouped['economy'] = grouped['economy'].round(2)
grouped['dot_ball_percentage'] = grouped['dot_ball_percentage'].round(2)
grouped['bowling_average'] = grouped['bowling_average'].round(2)

# Step 5: Reorder and rename columns
final_stats = grouped[[
    'overs_bowled', 'balls_bowled', 'total_runs_with_extras', 'wickets_taken',
    'economy', 'fours_given', 'sixes_given', 'dot_ball_percentage', 'bowling_average'
]]

final_stats.columns = [
    'Overs', 'Balls', 'Runs', 'Wkts', 'Econ', '4s', '6s', 'Dot%', 'Avg'
]

# Reset index to make bowler a column
final_stats = final_stats.reset_index()

# Show all columns in one row for top bowlers
print(final_stats.sort_values(by='Wkts', ascending=False).head(10).to_string(index=False))

        bowler  Overs  Balls  Runs  Wkts  Econ  4s  6s  Dot%   Avg
      DJ Bravo  161.3    969  1538   103  9.52 107  76 34.57 14.93
    SL Malinga  120.5    725   925    90  7.66  54  36 39.45 10.28
       B Kumar  152.3    915  1439    80  9.44 120  58 33.55 17.99
      HV Patel   88.4    532   875    68  9.87  54  54 39.47 12.87
     JJ Bumrah  127.2    764  1050    68  8.25  70  37 33.90 15.44
     MM Sharma   94.2    566   957    63 10.14  71  56 32.16 15.19
Mohammed Shami   73.4    442   798    57 10.83  59  46 31.00 14.00
     SP Narine   81.2    488   657    54  8.08  35  36 39.96 12.17
     CH Morris   72.3    435   594    50  8.19  38  23 35.86 11.88
 R Vinay Kumar   81.2    488   831    50 10.22  69  36 28.89 16.62


In [ ]:
# Filter only powerplay overs (1 to 6)
powerplay_batting = df[(df['over'] >= 1) & (df['over'] <= 6)]

# Group by batsman and compute stats
batter_stats = powerplay_batting.groupby('batsman').agg(
    Runs=('batsman_runs', 'sum'),
    Balls_Faced=('batsman_runs', 'count'),
    Fours=('batsman_runs', lambda x: (x == 4).sum()),
    Sixes=('batsman_runs', lambda x: (x == 6).sum()),
    Dismissals=('player_dismissed', lambda x: x.notna().sum())
).reset_index()

# Convert Fours and Sixes to integers
batter_stats['Fours'] = batter_stats['Fours'].astype(int)
batter_stats['Sixes'] = batter_stats['Sixes'].astype(int)

# Add computed metrics
batter_stats['Strike Rate'] = (batter_stats['Runs'] / batter_stats['Balls_Faced']) * 100
batter_stats['Boundary Rate'] = (batter_stats['Balls_Faced'] / (batter_stats['Fours'] + batter_stats['Sixes']))
batter_stats['Average'] = batter_stats.apply(
    lambda row: row['Runs'] / row['Dismissals'] if row['Dismissals'] > 0 else row['Runs'], axis=1
)

# Round values
batter_stats = batter_stats.round(2)

# Sort by Runs and select top 15
Batting_p = batter_stats.sort_values(by='Runs', ascending=False)

# ✅ Add Phase column
Batting_p['Phase'] = 'Powerplay'

Batting_p


,batsman,Runs,Balls_Faced,Fours,Sixes,Dismissals,Strike Rate,Boundary Rate,Average,Phase
277,S Dhawan,3320,2726,416,78,100,121.79,5.52,33.20,Powerplay
77,DA Warner,3171,2233,395,109,63,142.01,4.43,50.33,Powerplay
343,V Kohli,2699,2344,303,64,70,115.15,6.39,38.56,Powerplay
68,CH Gayle,2314,1667,239,153,68,138.81,4.25,34.03,Powerplay
105,G Gambhir,2244,1834,299,35,70,122.36,5.49,32.06,Powerplay
...,...,...,...,...,...,...,...,...,...,...
129,JC Archer,0,4,0,0,1,0.00,inf,0.00,Powerplay
173,Liton Das,0,1,0,0,1,0.00,inf,0.00,Powerplay
325,Sumit Kumar,0,3,0,0,0,0.00,inf,0.00,Powerplay
92,DL Chahar,0,2,0,0,0,0.00,inf,0.00,Powerplay


In [ ]:
# Convert 'over' to numeric if it's not already
df['over'] = pd.to_numeric(df['over'], errors='coerce')

# Step 1: Filter Powerplay overs (1 to 6)
powerplay_bowling = df[(df['over'] >= 1) & (df['over'] <= 6)]

# Step 2: Group by bowler and calculate basic stats
grouped = powerplay_bowling.groupby('bowler').agg(
    balls_bowled=('bowler', 'size'),
    runs_given=('batsman_runs', 'sum'),
    extras=('extras', 'sum'),
    wickets_taken=('player_dismissed', lambda x: x.notnull().sum()),
    fours_given=('batsman_runs', lambda x: x[x == 4].count()),
    sixes_given=('batsman_runs', lambda x: x[x == 6].count()),
    dot_balls=('batsman_runs', lambda x: x[x == 0].count())
)

# Step 3: Derived metrics
grouped['total_runs_with_extras'] = grouped['runs_given'] + grouped['extras']

# Overs Bowled (like 4.3)
grouped['overs_bowled'] = grouped['balls_bowled'] // 6 + (grouped['balls_bowled'] % 6) / 10

# Economy rate
grouped['economy'] = (grouped['total_runs_with_extras'] / grouped['balls_bowled']) * 6

# Dot ball percentage
grouped['dot_ball_percentage'] = (grouped['dot_balls'] / grouped['balls_bowled']) * 100

# Bowling average
grouped['bowling_average'] = grouped['total_runs_with_extras'] / grouped['wickets_taken'].replace(0, np.nan)

# Step 4: Round to 2 decimal places
grouped['economy'] = grouped['economy'].round(2)
grouped['dot_ball_percentage'] = grouped['dot_ball_percentage'].round(2)
grouped['bowling_average'] = grouped['bowling_average'].round(2)

# Step 5: Reorder and rename columns
Bowling_p = grouped[[
    'overs_bowled', 'balls_bowled', 'total_runs_with_extras', 'wickets_taken',
    'economy', 'fours_given', 'sixes_given', 'dot_ball_percentage', 'bowling_average'
]]

Bowling_p.columns = [
    'Overs', 'Balls', 'Runs', 'Wkts', 'Econ', '4s', '6s', 'Dot%', 'Avg'
]

# Reset index to make bowler a column
Bowling_p = final_stats.reset_index()
Bowling_p['Phase'] = 'Powerplay'

# Show all columns in one row for top bowlers
print(Bowling_p.sort_values(by='Wkts', ascending=False).to_string(index=False))

 index              bowler  Overs  Balls  Runs  Wkts  Econ  4s  6s   Dot%    Avg     Phase
    95            DJ Bravo  196.3   1179  1842   115  9.37 129  88  34.35  16.02 Powerplay
   341          SL Malinga  160.5    965  1252   104  7.78  81  41  37.72  12.04 Powerplay
    58             B Kumar  205.1   1231  1884    93  9.18 159  70  33.14  20.26 Powerplay
   151           JJ Bumrah  193.5   1163  1601    89  8.26 119  56  35.25  17.99 Powerplay
   124            HV Patel  110.4    664  1069    74  9.66  67  62  37.80  14.45 Powerplay
   228           MM Sharma  115.2    692  1142    72  9.90  84  61  30.35  15.86 Powerplay
   238      Mohammed Shami   97.5    587  1030    69 10.53  76  59  31.01  14.93 Powerplay
   348           SP Narine  125.5    755  1003    69  7.97  56  54  38.68  14.54 Powerplay
    76           CH Morris  101.3    609   862    61  8.49  61  33  33.99  14.13 Powerplay
   290       R Vinay Kumar   99.4    598  1022    58 10.25  80  48  28.93  17.62 Powerplay

In [ ]:
# Filter only powerplay overs (1 to 6)
Middleovers_batting = df[(df['over'] >= 7) & (df['over'] <= 15)]

# Group by batsman and compute stats
batter_stats = Middleovers_batting.groupby('batsman').agg(
    Runs=('batsman_runs', 'sum'),
    Balls_Faced=('batsman_runs', 'count'),
    Fours=('batsman_runs', lambda x: (x == 4).sum()),
    Sixes=('batsman_runs', lambda x: (x == 6).sum()),
    Dismissals=('player_dismissed', lambda x: x.notna().sum())
).reset_index()

# Convert Fours and Sixes to integers
batter_stats['Fours'] = batter_stats['Fours'].astype(int)
batter_stats['Sixes'] = batter_stats['Sixes'].astype(int)

# Add computed metrics
batter_stats['Strike Rate'] = (batter_stats['Runs'] / batter_stats['Balls_Faced']) * 100
batter_stats['Boundary Rate'] = (batter_stats['Balls_Faced']/(batter_stats['Fours'] + batter_stats['Sixes']))
batter_stats['Average'] = batter_stats.apply(
    lambda row: row['Runs'] / row['Dismissals'] if row['Dismissals'] > 0 else row['Runs'], axis=1
)

# Round values
batter_stats = batter_stats.round(2)

# Sort by Runs and select top 15
top_15 = batter_stats.sort_values(by='Runs', ascending=False)
# ✅ Add Phase column
top_15['Phase'] = 'Middleovers'

Batting_m = top_15
Batting_m

,batsman,Runs,Balls_Faced,Fours,Sixes,Dismissals,Strike Rate,Boundary Rate,Average,Phase
516,V Kohli,3737,2930,260,127,96,127.54,7.57,38.93,Middleovers
391,RG Sharma,3017,2466,246,99,96,122.34,7.15,31.43,Middleovers
444,SK Raina,2999,2260,227,114,97,132.70,6.63,30.92,Middleovers
25,AB de Villiers,2808,2032,191,120,62,138.19,6.53,45.29,Middleovers
229,KD Karthik,2696,2211,248,62,103,121.94,7.13,26.17,Middleovers
...,...,...,...,...,...,...,...,...,...,...
476,Sanvir Singh,0,1,0,0,1,0.00,inf,0.00,Middleovers
496,T Kohli,0,1,0,0,0,0.00,inf,0.00,Middleovers
495,T Kohler-Cadmore,0,1,0,0,1,0.00,inf,0.00,Middleovers
491,Sunny Gupta,0,1,0,0,1,0.00,inf,0.00,Middleovers


In [ ]:
# Convert 'over' to numeric if it's not already
df['over'] = pd.to_numeric(df['over'], errors='coerce')

# Step 1: Filter Powerplay overs (1 to 6)
Middleover_bowling = df[(df['over'] >= 7) & (df['over'] <= 15)]

# Step 2: Group by bowler and calculate basic stats
grouped = Middleover_bowling.groupby('bowler').agg(
    balls_bowled=('bowler', 'size'),
    runs_given=('batsman_runs', 'sum'),
    extras=('extras', 'sum'),
    wickets_taken=('player_dismissed', lambda x: x.notnull().sum()),
    fours_given=('batsman_runs', lambda x: x[x == 4].count()),
    sixes_given=('batsman_runs', lambda x: x[x == 6].count()),
    dot_balls=('batsman_runs', lambda x: x[x == 0].count())
)

# Step 3: Derived metrics
grouped['total_runs_with_extras'] = grouped['runs_given'] + grouped['extras']

# Overs Bowled (like 4.3)
grouped['overs_bowled'] = grouped['balls_bowled'] // 6 + (grouped['balls_bowled'] % 6) / 10

# Economy rate
grouped['economy'] = (grouped['total_runs_with_extras'] / grouped['balls_bowled']) * 6

# Dot ball percentage
grouped['dot_ball_percentage'] = (grouped['dot_balls'] / grouped['balls_bowled']) * 100

# Bowling average
grouped['bowling_average'] = grouped['total_runs_with_extras'] / grouped['wickets_taken'].replace(0, np.nan)

# Step 4: Round to 2 decimal places
grouped['economy'] = grouped['economy'].round(2)
grouped['dot_ball_percentage'] = grouped['dot_ball_percentage'].round(2)
grouped['bowling_average'] = grouped['bowling_average'].round(2)

# Step 5: Reorder and rename columns
Bowling_m = grouped[[
    'overs_bowled', 'balls_bowled', 'total_runs_with_extras', 'wickets_taken',
    'economy', 'fours_given', 'sixes_given', 'dot_ball_percentage', 'bowling_average'
]]

Bowling_m.columns = [
    'Overs', 'Balls', 'Runs', 'Wkts', 'Econ', '4s', '6s', 'Dot%', 'Avg'
]

# Reset index to make bowler a column
Bowling_m = Bowling_m.reset_index()

# ✅ Add Phase column
Bowling_m['Phase'] = 'Middleovers'

# Show all columns in one row for top bowlers
print(Bowling_m.sort_values(by='Wkts', ascending=False).to_string(index=False))

                 bowler  Overs  Balls  Runs  Wkts  Econ  4s  6s  Dot%    Avg       Phase
              YS Chahal  420.4   2524  3192   144  7.59 169 150 36.89  22.17 Middleovers
              PP Chawla  480.5   2885  3810   140  7.92 245 171 34.80  27.21 Middleovers
               A Mishra  444.4   2668  3185   130  7.16 169 137 36.73  24.50 Middleovers
               R Ashwin  469.0   2814  3292   121  7.02 169 121 34.51  27.21 Middleovers
              RA Jadeja  480.2   2882  3600   120  7.49 206 150 33.62  30.00 Middleovers
            Rashid Khan  346.1   2077  2320   104  6.70 149  81 38.76  22.31 Middleovers
              SP Narine  364.3   2187  2348   101  6.44 127  71 38.23  23.25 Middleovers
        Harbhajan Singh  373.5   2243  2659    95  7.11 169  92 35.76  27.99 Middleovers
               AR Patel  353.0   2118  2558    82  7.25 137 105 33.33  31.20 Middleovers
               DJ Bravo  285.3   1713  2127    70  7.45 174  52 35.55  30.39 Middleovers
          Kuldeep Yad

In [ ]:
# Filter only Death overs (16 to 20)
Deathovers_df = df[(df['over'] >= 16) & (df['over'] <= 20)]

# Group by batsman and compute stats
death_batting = Deathovers_df.groupby('batsman').agg(
    Runs=('batsman_runs', 'sum'),
    Balls_Faced=('batsman_runs', 'count'),
    Fours=('batsman_runs', lambda x: (x == 4).sum()),
    Sixes=('batsman_runs', lambda x: (x == 6).sum()),
    Dismissals=('player_dismissed', lambda x: x.notna().sum())
).reset_index()

# Convert Fours and Sixes to integers
death_batting['Fours'] = death_batting['Fours'].astype(int)
death_batting['Sixes'] = death_batting['Sixes'].astype(int)

# Add computed metrics
death_batting['Strike Rate'] = (death_batting['Runs'] / death_batting['Balls_Faced']) * 100
death_batting['Boundary Rate'] = death_batting['Balls_Faced'] / (death_batting['Fours'] + death_batting['Sixes'])
death_batting['Average'] = death_batting.apply(
    lambda row: row['Runs'] / row['Dismissals'] if row['Dismissals'] > 0 else row['Runs'], axis=1
)

# Round values
death_batting = death_batting.round(2)

# Sort by Runs and select top 15
top_15 = death_batting.sort_values(by='Runs', ascending=False)

# ✅ Add Phase column
top_15['Phase'] = 'Deathovers'

Batting_d = top_15
Batting_d


,batsman,Runs,Balls_Faced,Fours,Sixes,Dismissals,Strike Rate,Boundary Rate,Average,Phase
328,MS Dhoni,2786,1556,211,175,91,179.05,4.03,30.62,Deathovers
242,KA Pollard,1708,994,110,127,76,171.83,4.19,22.47,Deathovers
247,KD Karthik,1565,881,142,91,72,177.64,3.78,21.74,Deathovers
24,AB de Villiers,1421,635,106,112,35,223.78,2.91,40.60,Deathovers
416,RA Jadeja,1420,931,104,69,66,152.52,5.38,21.52,Deathovers
...,...,...,...,...,...,...,...,...,...,...
455,S Ladda,0,1,0,0,1,0.00,inf,0.00,Deathovers
58,Abdur Razzak,0,2,0,0,0,0.00,inf,0.00,Deathovers
518,Shoaib Ahmed,0,4,0,0,2,0.00,inf,0.00,Deathovers
546,U Kaul,0,1,0,0,0,0.00,inf,0.00,Deathovers


In [ ]:
# Convert 'over' to numeric if it's not already
df['over'] = pd.to_numeric(df['over'], errors='coerce')

# Step 1: Filter Powerplay overs (1 to 6)
death_bowling = df[(df['over'] >= 16) & (df['over'] <= 20)]

# Step 2: Group by bowler and calculate basic stats
grouped = death_bowling.groupby('bowler').agg(
    balls_bowled=('bowler', 'size'),
    runs_given=('batsman_runs', 'sum'),
    extras=('extras', 'sum'),
    wickets_taken=('player_dismissed', lambda x: x.notnull().sum()),
    fours_given=('batsman_runs', lambda x: x[x == 4].count()),
    sixes_given=('batsman_runs', lambda x: x[x == 6].count()),
    dot_balls=('batsman_runs', lambda x: x[x == 0].count())
)

# Step 3: Derived metrics
grouped['total_runs_with_extras'] = grouped['runs_given'] + grouped['extras']

# Overs Bowled (like 4.3)
grouped['overs_bowled'] = grouped['balls_bowled'] // 6 + (grouped['balls_bowled'] % 6) / 10

# Economy rate
grouped['economy'] = (grouped['total_runs_with_extras'] / grouped['balls_bowled']) * 6

# Dot ball percentage
grouped['dot_ball_percentage'] = (grouped['dot_balls'] / grouped['balls_bowled']) * 100

# Bowling average
grouped['bowling_average'] = grouped['total_runs_with_extras'] / grouped['wickets_taken'].replace(0, np.nan)

# Step 4: Round to 2 decimal places
grouped['economy'] = grouped['economy'].round(2)
grouped['dot_ball_percentage'] = grouped['dot_ball_percentage'].round(2)
grouped['bowling_average'] = grouped['bowling_average'].round(2)

# Step 5: Reorder and rename columns
final_stats = grouped[[
    'overs_bowled', 'balls_bowled', 'total_runs_with_extras', 'wickets_taken',
    'economy', 'fours_given', 'sixes_given', 'dot_ball_percentage', 'bowling_average'
]]

final_stats.columns = [
    'Overs', 'Balls', 'Runs', 'Wkts', 'Econ', '4s', '6s', 'Dot%', 'Avg'
]

# Reset index to make bowler a column
final_stats = final_stats.reset_index()
final_stats['Phase']='Deathovers'
Bowling_d = final_stats
# Show all columns in one row for top bowlers
print(Bowling_d.sort_values(by='Wkts', ascending=False).to_string(index=False))

             bowler  Overs  Balls  Runs  Wkts  Econ  4s  6s   Dot%    Avg      Phase
           DJ Bravo  196.3   1179  1842   115  9.37 129  88  34.35  16.02 Deathovers
         SL Malinga  160.5    965  1252   104  7.78  81  41  37.72  12.04 Deathovers
            B Kumar  205.1   1231  1884    93  9.18 159  70  33.14  20.26 Deathovers
          JJ Bumrah  193.5   1163  1601    89  8.26 119  56  35.25  17.99 Deathovers
           HV Patel  110.4    664  1069    74  9.66  67  62  37.80  14.45 Deathovers
          MM Sharma  115.2    692  1142    72  9.90  84  61  30.35  15.86 Deathovers
     Mohammed Shami   97.5    587  1030    69 10.53  76  59  31.01  14.93 Deathovers
          SP Narine  125.5    755  1003    69  7.97  56  54  38.68  14.54 Deathovers
          CH Morris  101.3    609   862    61  8.49  61  33  33.99  14.13 Deathovers
      R Vinay Kumar   99.4    598  1022    58 10.25  80  48  28.93  17.62 Deathovers
           UT Yadav  109.2    656  1144    56 10.46  90  60  31.7

In [ ]:
# Combine all three batting phase tables
final_batting_df = pd.concat([Batting_p, Batting_m, Batting_d], ignore_index=True)

# Optional: Display the final combined batting DataFrame
final_batting_df


,batsman,Runs,Balls_Faced,Fours,Sixes,Dismissals,Strike Rate,Boundary Rate,Average,Phase
0,S Dhawan,3320,2726,416,78,100,121.79,5.52,33.20,Powerplay
1,DA Warner,3171,2233,395,109,63,142.01,4.43,50.33,Powerplay
2,V Kohli,2699,2344,303,64,70,115.15,6.39,38.56,Powerplay
3,CH Gayle,2314,1667,239,153,68,138.81,4.25,34.03,Powerplay
4,G Gambhir,2244,1834,299,35,70,122.36,5.49,32.06,Powerplay
...,...,...,...,...,...,...,...,...,...,...
1493,S Ladda,0,1,0,0,1,0.00,inf,0.00,Deathovers
1494,Abdur Razzak,0,2,0,0,0,0.00,inf,0.00,Deathovers
1495,Shoaib Ahmed,0,4,0,0,2,0.00,inf,0.00,Deathovers
1496,U Kaul,0,1,0,0,0,0.00,inf,0.00,Deathovers


In [ ]:
# Save the final batting dataset to a CSV file
final_batting_df.to_csv('/content/final_batting_dataset.csv', index=False)

# Provide a download link
from google.colab import files
files.download('/content/final_batting_dataset.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>